In [137]:
%run -n main.py
dotenv = dict(read_dotenv('.env'))
openai.api_key = dotenv['OPENAI_TOKEN']

In [ ]:
items = read_json('data/classify.json')
for item in items:
    tags = j

# sources

## alpaca

In [ ]:
!mkdir -p data/sources/alpaca
!curl -L https://github.com/yizhongw/self-instruct/raw/main/human_eval/user_oriented_instructions.jsonl \
    > data/sources/alpaca/user_oriented_instructions.jsonl

In [ ]:
%run -n main.py
items = read_jsonl('data/sources/alpaca/user_oriented_instructions.jsonl')
alpaca_items = list(parse_alpaca(items))

## vicuna

In [ ]:
!mkdir -p data/sources/vicuna
!curl -L https://github.com/lm-sys/vicuna-blog-eval/raw/main/eval/table/question.jsonl \
    > data/sources/vicuna/question.jsonl

In [ ]:
%run -n main.py
items = read_jsonl('data/sources/vicuna/question.jsonl')
vicuna_items = list(parse_vicuna(items))

## arena

In [ ]:
!mkdir -p data/sources/arena
!curl -L curl -L https://huggingface.co/datasets/lmsys/chatbot_arena_conversations/resolve/main/data/train-00000-of-00001-cced8514c7ed782a.parquet \
    > data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet

In [ ]:
%run -n main.py
records = pd.read_parquet('data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet').itertuples()
arena_items = list(parse_arena(records))

# orig

In [ ]:
orig_items = alpaca_items + vicuna_items

In [ ]:
instruction_items = {
    _['instruction']: _
    for _ in arena_items
    if _['lang'] == 'English'
}
orig_items.extend(random.sample(list(instruction_items.values()), 1000))

In [14]:
write_json('data/orig.json', orig_items)

In [15]:
orig_items = read_json('data/orig.json')
random.sample(orig_items, 5)

[{'id': 'c189117c-b2a1-4ac9-9f0e-06370cfe8b6b',
  'source': 'arena',
  'source_id': 'adccaf90b7904fdeb56a1da9150c2492',
  'lang': 'English',
  'instruction': 'Let\'s think about writing a Python script step by step. \n\n1) Analyze and define the use case. What does the script need to accomplish? \n\n2) Analyze the required parameters? What does it need to know? How can it get the parameters? \n\n3) Define functions for use cases.\n\n4) Execute the script if it is running as the main module.\n\nHere is an example script for saving a message to a file:\n\nimport os\n\ndef get_user_input():\n    message = input("Please enter your message: ")\n    file_name = input("Please enter the file name: ")\n    return message, file_name\n\ndef save_message_to_file(message, file_name):\n    with open(file_name, \'w\') as file:\n        file.write(message)\n    print(f"Message saved to {file_name}")\n\ndef main():\n    message, file_name = get_user_input()\n    save_message_to_file(message, file_name)

# tasks

In [21]:
task_items = read_json('data/tasks.json')
id_task_items 

# translate

In [ ]:
translate_items = [
    {
        'id': _['id'],
        'instruction': _['instruction'],
        'answer': None
    }
    for _ in orig_items
]

In [ ]:
%run -n main.py
items = [_ for _ in translate_items if not _['answer']]
queue = iter(tqdm(items))
workers = [openai_translate_worker(queue) for _ in range(10)]
await asyncio.gather(*workers);

In [17]:
write_json('data/translate.json', translate_items)

# label studio

In [ ]:
%run -n main.py
label_studio = label_studio_sdk.Client('http://localhost:8080', dotenv['LABELSTUDIO_TOKEN'])
label_studio.check_connection()

In [ ]:
title_projects = {
    _.title: _
    for _ in label_studio.list_projects()
}
translate_project = title_projects['translate']
classify_project = title_projects['classify']

# translate annot

In [ ]:
translate_items = read_json('data/translate.json')
annot_items = [translate_annot_item(_) for _ in translate_items]
random.choice(annot_items)

In [ ]:
translate_project.delete_all_tasks();
translate_project.import_tasks(annot_items);

In [ ]:
annot_items = translate_project.export_tasks()
translate_items = [annot_translate_item(_) for _ in annot_items]
random.sample(translate_items, 3)

In [ ]:
write_json('data/translate.json', translate_items)

# classify

In [ ]:
%run -n main.py
classify_items = read_json('data/classify.json')
id_embeddings = read_pickle('data/embeddings.pkl')

In [ ]:
%run -n main.py
items = [
    _ for _ in classify_items
    if _['id'] not in id_embeddings
]
for index in tqdm(range(0, len(items), 64)):
    batch = items[index:index + 64]
    texts = [_['instruction'] for _ in batch]
    embeddings = openai_embed_batch(texts)
    for item, embedding in zip(batch, embeddings):
        id_embeddings[item['id']] = np.array(embedding)
write_pickle('data/embeddings.pkl', id_embeddings)

In [ ]:
target_items = [
    _ for _ in classify_items
    if _['category'] and _['category'] != 'bad instruction'
]
items = [_ for _ in classify_items if not _.category]

for item in tqdm(items):
    max_sim = 0
    for target_item in target_items:
        sim = cosine_sim(
            id_embeddings[item['id']],
            id_embeddings[target_item['id']]
        )
        if sim > max_sim:
            max_sim = sim
            item['category'] = target_item['category']
    item['max_sim'] = max_sim

In [ ]:
items = []
for item in classify_items:
    if not item.get('max_sim'):
        continue
        
    if item['category'] != 'enumerate':
        continue
        
    items.append(item)

items = sorted(items, key=lambda _: _['max_sim'], reverse=False)
annot_items = [classify_annot_item(_) for _ in items]
len(annot_items)

In [ ]:
classify_project.delete_all_tasks();
classify_project.import_tasks(annot_items);

In [ ]:
%run -n main.py
annot_items = classify_project.export_tasks()
items = (annot_classify_item(_) for _ in annot_items)
id_categories = {
    _['id']: _['category']
    for _ in items
}
for item in classify_items:
    category = id_categories.get(item['id'])
    if category is not None:
        item['category'] = category
        item.pop('max_sim', None)

In [ ]:
for item in classify_items:
    if item.pop('max_sim', None):
        item['category'] = []
write_json('data/classify.json', classify_items)

# tasks

In [13]:
items = read_json('data/classify.json')
id_categories = {_['id']: _['category'] for _ in items if _['category']}
len(id_categories)

551

In [14]:
items = read_json('data/orig.json')
id_sources = {_['id']: _['source'] for _ in items}
len(id_sources)

1317

In [16]:
task_items = []
items = read_json('data/translate.json')
for item in items:
    id = item['id']
    category = id_categories.get(id)
    if category == 'bad instruction':
        continue

    task_items.append({
        'id': id,
        'source': id_sources[id],
        'instruction': item['answer'],
        'category': category
    })
write_json('data/tasks.json', task_items)

In [12]:
task_items = read_json('data/tasks.json')

In [27]:
%run -n main.py
source_items = defaultdict(list)
for item in task_items:
    source_items[item['source']].append(item)

with open('data/tasks.md', 'w') as file:
    with redirect_stdout(file):
        for source in ['alpaca', 'vicuna', 'arena']:
            print(f'<h1>{source}</h1>')

            items = [_ for _ in source_items[source] if _['category']]
            for item in random.sample(items, 30):
                print('<code>#{_}</code>' % item['category']))
                print('<br/>')
                instruction = html.escape(item['instruction'])
                print('<br/>\n'.join(instruction.splitlines()))
                print('<br/><br/>')

# infer

In [273]:
%run -n main.py
infer_items = read_json('data/infer.json')

In [279]:
infer_items.extend(
    {
        'id': _['id'],
        'model': 'yagpt_chat',
        'instruction': _['instruction'],
        'answer': None
    }
    for _ in task_items
)

In [295]:
write_json('data/infer.json', infer_items)

In [ ]:
items = [_ for _ in infer_items if _['model'] == 'yagpt_chat' and _['answer'] is not None]
print('|items| =', len(items))

# random.shuffle(items)
for item in items[-10:]:
    if item['answer']:
        print(item['instruction'])
        print('----')
        print(item['answer'])
        print('---\n\n')

## turbo

In [ ]:
%run -n main.py
items = [_ for _ in infer_items if _['answer'] is None and _['model'] == 'turbo_2']
queue = iter(tqdm(items))
workers = [openai_infer_worker(queue, model='gpt-3.5-turbo-0613') for _ in range(20)]
await asyncio.gather(*workers);

## gpt4

In [ ]:
%run -n main.py
items = [_ for _ in infer_items if _['answer'] is None and _['model'] == 'gpt4_2']
queue = iter(tqdm(items))
workers = [openai_infer_worker(queue, model='gpt-4-0613', request_timeout=1200) for _ in range(20)]
await asyncio.gather(*workers);

## gigachat

In [74]:
%run -n main.py
headers = dict(read_headers('.gigachat'))
gigachat_client = gigachat_client_init(headers)

# After ~5 min / 260 answers blocked for ~1 hour
# {'result': 'rejected', 'reason': 'UserBlocked', 'user_blocked_until': '2023-08-25T11:00:24+00:00'}

# "в полуавтоматическом режиме банят, если 3 временных бана, то могут опять решить забанить насовсем.
# Так что при временном бане лучше какое-то время  подождать."

# "из-за запросов типа "Люди умирают, когда их убивают, откуда это высказывание?". Цензор такое
# отлавливает сколько-то раз и во временный бан отправляет"

In [ ]:
%run -n main.py
items = [_ for _ in infer_items if _['answer'] is None and _['model'] == 'gigachat']
queue = iter(tqdm(items[:100]))
workers = [gigachat_infer_worker(gigachat_client, queue) for _ in range(2)]
await asyncio.gather(*workers);

## yagpt

In [162]:
lines = !~/yandex-cloud/bin/yc iam create-token
YAGPT_TOKEN = lines[0]

lines = !~/yandex-cloud/bin/yc resource-manager folder get --name default --format json
data = json.loads(''.join(lines))
YAGPT_FOLDER_ID = data['id']

# token expires every ~12 hours

In [ ]:
%run -n main.py
yagpt_client = yagpt_client_init(YAGPT_TOKEN, YAGPT_FOLDER_ID)

In [ ]:
%run -n main.py
items = [_ for _ in infer_items if _['answer'] is None and _['model'] == 'yagpt_instruct']
queue = iter(tqdm(items))
limiter = Limiter(min_delay=1.2)
workers = [yagpt_infer_worker(yagpt_client, limiter, queue, mode='instruct') for _ in range(5)]
await asyncio.gather(*workers);

In [ ]:
%run -n main.py
items = [_ for _ in infer_items if _['answer'] is None and _['model'] == 'yagpt_chat']
queue = iter(tqdm(items))
limiter = Limiter(min_delay=1.2)
workers = [yagpt_infer_worker(yagpt_client, limiter, queue, mode='chat') for _ in range(5)]
await asyncio.gather(*workers);